**Defines and imports**

In [1]:
import json
import re
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

PATH = "../shared/scenarios/3n_delayed/results-3n-delayed-1678894718/"

with open(PATH + "start.txt") as file:
    simStart = int(file.readline().strip())
with open(PATH + "stop.txt") as file:
    simStop = int(file.readline().strip())


**Resource Graphs**

In [36]:
def plotResourceGraphs():
    logfiles = []

    for root, _, files in os.walk(PATH):
        for file in files:
            if file in ["pidstat-base.csv.log", "pidstat-robot.csv.log"]:
                logfiles.append(root + "/" + file)

    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Base", "Robot"),
        specs=[[{"secondary_y": True}, {"secondary_y": True}]],
    )
    fig.update_xaxes(title_text="Time [s]")
    fig.update_yaxes(
        title_text="RSS [kB]",
        secondary_y=False,
        titlefont=dict(color="#ab63fa"),
        tickfont=dict(color="#ab63fa"),
    )
    fig.update_yaxes(
        title_text="CPU Usage [%]",
        secondary_y=True,
        titlefont=dict(color="#00cc96"),
        tickfont=dict(color="#00cc96"),
    )
    fig.update_layout(title_text="Resource Usage")

    for file in logfiles:
        # Get node name
        nodeName = re.search(r"/pidstat-(\w*)", file).group(1)
        column = 1 if ("base" in nodeName) else 2

        # Get number of CPU Cores
        with open(file.replace(".csv", "")) as f:
            cpuCores = int(re.search(r"\((\d*) CPU\)", f.readline()).group(1))

        df = pd.read_csv(file, sep="\s+", usecols=[0, 7, 11, 12])
        # substract start time
        df = df.subtract([simStart, 0, 0, 0], axis="columns")
        # devide CPU usage by core count
        df = df.divide([1, cpuCores, 1, 1], axis="columns")

        # plot
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["RSS"],
                mode="lines",
                name="RSS",
                line=dict(color="#ab63fa", width=2),
            ),
            secondary_y=False,
            row=1,
            col=column,
        )
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["%CPU"],
                mode="lines",
                name="CPU",
                line=dict(color="#00cc96", width=2),
            ),
            secondary_y=True,
            row=1,
            col=column,
        )

        # Print mean values
        stats = f"Mean Values {nodeName}:\nCPU: \t{df.loc[:,'%CPU'].mean()} %\nRSS: \t{df.loc[:,'RSS'].mean()} kB\n"
        print(stats)

    fig.show()

plotResourceGraphs()


Mean Values base:
CPU: 	0.0017605633802816902 %
RSS: 	24484.0 kB

Mean Values robot:
CPU: 	0.007042253521126761 %
RSS: 	25004.0 kB

